# Iron Supported, Nitrogen-doped, Graphene (And Related Systems) <a name="head"></a>
***

TODO:
* Include Vegge data in plot
* TEMP

# Import Modules <a name="import-modules"></a>

In [ ]:
%%capture
%load_ext autoreload
%autoreload 2

In [1]:
# Setting Custom Paths ********************************************************
# *****************************************************************************
import os
import sys

sys.path.insert(
    0, os.path.join(
        os.environ["PROJ_fe_graph"],
        "data"))

sys.path.insert(
    0, os.path.join(
        os.environ["PROJ_fe_graph"],
        "workflow"))


# Python Modules **************************************************************
# *****************************************************************************
import pandas as pd
# pd.options.mode.chained_assignment = None
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', None)

import plotly.graph_objs as go
import plotly.plotly as py
import numpy as np


# My Modules ******************************************************************
# *****************************************************************************
from orr_reaction.orr_fed_plot import ORR_Free_E_Plot, Scaling_Relations_Plot


# Local Imports ***************************************************************
# *****************************************************************************
from an_data_processing import load_df
from proj_data_fe_graph import proj_dir_name, system_color_map

# Script Inputs

In [2]:
save_plot = True
# save_plot = False

bias = 0.


# Smart Formatting
smart_format = [

    # [
    #     {"spinpol": True},
    #     {"dash": "dot"},
    #     ],

    [
        {"system": "Fe_slab"},
        {"dash": "dashdot"},
        ],

    [
        {"system": "N_graph_Fe"},
        {"dash": "dot"},
        ],

    [
        {"system": "graph_Fe"},
        {"dash": "dash"},
        ],

    [
        {"system": "graphene"},
        {"dash": None},
        ],

    ]


In [3]:
%%capture
df_m = load_df(
    from_file=False,
    root_dir=os.path.join(
        os.environ["PROJ_fe_graph"],
        "workflow/adsorption_study"),
    )

# Exlcuding pdos and charge history columns from df

In [4]:
all_columns = set(list(df_m))

exclude_columns = set([
    "pdos_data",
    "charge_history",
    ])

columns_to_keep = list(all_columns - exclude_columns)

df_m = df_m[columns_to_keep]

# Instantiate the ORR_Free_E_Plot Instance

In [5]:
prop_name_list = [
    "system",
    "spinpol",
    "site",
    "graph_site",
     ]

ORR_PLT = ORR_Free_E_Plot(
    free_energy_df=None,
    state_title="adsorbate",
    free_e_title="ads_e",  
    bias=bias,
    color_list=None,   
    show_H_e_pairs_annotations=True,
    show_legend=False,
    rxn_type="ORR",
    )

# Picking the Lowest Energy States

In [6]:
group_params = ["system", "spinpol", "graph_site"]
grouped = df_m.groupby(group_params)

lowest_energy_list = []
for i_ind, (name, group) in enumerate(grouped):
    df_i = group

    for j_cnt, df_j in df_i.groupby("adsorbate"):
        series_j = df_j.loc[df_j["ads_e"].idxmin()]
        lowest_energy_list.append(series_j)

df_2 = pd.concat(lowest_energy_list, axis=1, keys=[s.name for s in lowest_energy_list]).T

# Adding ORR data series

In [7]:
ORR_PLT.ideal_ORR_series()

In [8]:
grouped = df_2.groupby(group_params)
for i_ind, (name, group) in enumerate(grouped):
    df_i = group

    prop_list_i = {}
    for prop_i in prop_name_list:    
        first_value = df_i[prop_i].tolist()[0]
        all_same_value = all([True if i == first_value else False for i in df_i[prop_i].tolist()])

        if all_same_value:
            prop_list_i[prop_i] = str(first_value)
        else:
            prop_list_i[prop_i] = str(None)

    name_i = "_".join(list(prop_list_i.values()))
#     print(name_i)
    
    color_i = system_color_map.get(name_i, "pink")

    if not any([np.isnan(i) for i in df_i.elec_energy.tolist()]):        
        ORR_PLT.add_series(
            df_i,
            plot_mode="all",
            opt_name=name_i,
            overpotential_type="ORR",
            color=color_i,
            )

# Plotting <a name="fed_all"></a>

In [9]:
if save_plot:
    save_dir = proj_dir_name
else:
    save_dir = "__temp__"
# save_dir = "__temp__"

In [10]:
layout = ORR_PLT.plotly_fed_layout(
    plot_title="",
    plot_width=18.7 * 37.795275591,
    plot_height= 3. / 4. * 18.7 * 37.795275591,
    )

In [11]:
py.plotly.iplot(
    {
        "data": ORR_PLT.plotly_data(),
        "layout": layout,
        },
        filename=os.path.join(save_dir, "fed_plots", "fed_all_systems_main_no_legend"),
    )

In [12]:
layout["width"] = 18.7 * 37.795275591
layout["height"] = 6. / 4. * 18.7 * 37.795275591
layout["showlegend"] = True

py.plotly.iplot(
    {
        "data": ORR_PLT.plotly_data(),
        "layout": layout,
        },
        filename=os.path.join(save_dir, "fed_plots", "fed_all_systems_long_w_legend"),
    )